<a href="https://colab.research.google.com/github/Dwarampudivivek/unique_projects/blob/main/vacuum_world.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
class VacuumAgent:
    def __init__(self, environment):
        self.environment = environment
        self.position = (0, 0)
        self.cleaned = set()
        self.moves = []

    def perceive(self):
        x, y = self.position
        return self.environment[x][y] == 'D'

    def update_model(self):
        if self.perceive():
            self.cleaned.add(self.position)
            self.environment[self.position[0]][self.position[1]] = 'C'

    def decide(self):
        if self.perceive():
            return 'clean'
        else:

            x, y = self.position
            if y + 1 < len(self.environment[0]):
                self.position = (x, y + 1)
            elif x + 1 < len(self.environment):
                self.position = (x + 1, 0)
            return 'move'

    def act(self):
        action = self.decide()
        if action == 'clean':
            print(f"Cleaning position {self.position}")
        elif action == 'move':
            print(f"Moving to position {self.position}")
            self.moves.append(self.position)

    def run(self):
        print("Starting Environment:")
        self.print_environment()
        print("\nAgent Actions:")
        while not self.is_done():
            self.update_model()
            self.act()
            self.print_environment()
        print("\nCleaning complete.")

    def is_done(self):

        return all(cell == 'C' for row in self.environment for cell in row)

    def print_environment(self):
        for row in self.environment:
            print(' '.join(row))
        print()

environment = [
    ['D', 'C', 'C'],
    ['D', 'D', 'C'],
    ['C', 'C', 'C']
]

agent = VacuumAgent(environment)
agent.run()


Starting Environment:
D C C
D D C
C C C


Agent Actions:
Moving to position (0, 1)
C C C
D D C
C C C

Moving to position (0, 2)
C C C
D D C
C C C

Moving to position (1, 0)
C C C
D D C
C C C

Moving to position (1, 1)
C C C
C D C
C C C

Moving to position (1, 2)
C C C
C C C
C C C


Cleaning complete.


In [ ]:
class GoalBasedVacuumAgent:
    def __init__(self, environment):
        self.environment = environment
        self.position = (0, 0)
        self.cleaned = set()
        self.goal = self.get_dirty_positions()

    def get_dirty_positions(self):
        return {(x, y) for x, row in enumerate(self.environment) for y, cell in enumerate(row) if cell == 'D'}

    def perceive(self):
        x, y = self.position
        return self.environment[x][y] == 'D'

    def update_model(self):
        if self.perceive():
            self.cleaned.add(self.position)
            self.environment[self.position[0]][self.position[1]] = 'C'
            self.goal.discard(self.position)

    def decide(self):
        if not self.goal:
            return 'done'


        x, y = self.position
        nearest_dirty = min(self.goal, key=lambda pos: abs(pos[0] - x) + abs(pos[1] - y))
        return self.move_towards(nearest_dirty)

    def move_towards(self, target):
        x, y = self.position
        tx, ty = target
        if x < tx:
            return 'move_down'
        elif x > tx:
            return 'move_up'
        elif y < ty:
            return 'move_right'
        elif y > ty:
            return 'move_left'

    def act(self):
        action = self.decide()
        if action == 'clean':
            print(f"Cleaning position {self.position}")
        elif action == 'move_down':
            self.position = (self.position[0] + 1, self.position[1])
            print(f"Moving to position {self.position}")
        elif action == 'move_up':
            self.position = (self.position[0] - 1, self.position[1])
            print(f"Moving to position {self.position}")
        elif action == 'move_right':
            self.position = (self.position[0], self.position[1] + 1)
            print(f"Moving to position {self.position}")
        elif action == 'move_left':
            self.position = (self.position[0], self.position[1] - 1)
            print(f"Moving to position {self.position}")
        elif action == 'done':
            print("All goals achieved. Cleaning complete.")

    def run(self):
        print("Starting Environment:")
        self.print_environment()
        print("\nAgent Actions:")
        while self.goal:
            self.update_model()
            self.act()
            self.print_environment()
        print("\nCleaning complete.")

    def print_environment(self):
        for row in self.environment:
            print(' '.join(row))
        print()


environment = [
    ['D', 'C', 'C'],
    ['D', 'D', 'C'],
    ['C', 'C', 'C']
]

agent = GoalBasedVacuumAgent(environment)
agent.run()


Starting Environment:
D C C
D D C
C C C


Agent Actions:
Moving to position (1, 0)
C C C
D D C
C C C

Moving to position (1, 1)
C C C
C D C
C C C

All goals achieved. Cleaning complete.
C C C
C C C
C C C


Cleaning complete.


In [ ]:

import random

class UtilityBasedVacuumAgent:
    def __init__(self, environment):
        self.environment = environment
        self.position = (0, 0)
        self.cleaned = set()
        self.utility = 0

    def perceive(self):
        x, y = self.position
        return self.environment[x][y] == 'D'

    def update_model(self):
        if self.perceive():
            self.cleaned.add(self.position)
            self.environment[self.position[0]][self.position[1]] = 'C'
            self.utility += 1

    def decide(self):
        if self.is_done():
            return 'done'


        potential_moves = []
        x, y = self.position
        directions = [('move_right', (x, y + 1)), ('move_down', (x + 1, y)),
                      ('move_left', (x, y - 1)), ('move_up', (x - 1, y))]

        for direction, (nx, ny) in directions:
            if 0 <= nx < len(self.environment) and 0 <= ny < len(self.environment[0]):

                potential_utilities = sum(1 for (dx, dy) in [(nx, ny)] if self.environment[dx][dy] == 'D')
                potential_moves.append((potential_utilities, direction, (nx, ny)))


        potential_moves.sort(reverse=True)
        if potential_moves:
            _, best_move, new_position = potential_moves[0]
            return best_move
        return 'done'

    def act(self):
        action = self.decide()
        if action == 'clean':
            print(f"Cleaning position {self.position}")
        elif action == 'move_right':
            self.position = (self.position[0], self.position[1] + 1)
            print(f"Moving to position {self.position}")
        elif action == 'move_down':
            self.position = (self.position[0] + 1, self.position[1])
            print(f"Moving to position {self.position}")
        elif action == 'move_left':
            self.position = (self.position[0], self.position[1] - 1)
            print(f"Moving to position {self.position}")
        elif action == 'move_up':
            self.position = (self.position[0] - 1, self.position[1])
            print(f"Moving to position {self.position}")
        elif action == 'done':
            print("All goals achieved. Cleaning complete.")

    def is_done(self):

        return all(cell == 'C' for row in self.environment for cell in row)

    def run(self):
        print("Starting Environment:")
        self.print_environment()
        print("\nAgent Actions:")
        while not self.is_done():
            self.update_model()
            self.act()
            self.print_environment()
        print("\nCleaning complete.")

    def print_environment(self):
        for row in self.environment:
            print(' '.join(row))
        print()


environment = [
    ['D', 'C', 'C'],
    ['D', 'D', 'C'],
    ['C', 'C', 'C']
]

agent = UtilityBasedVacuumAgent(environment)
agent.run()


Starting Environment:
D C C
D D C
C C C


Agent Actions:
Moving to position (1, 0)
C C C
D D C
C C C

Moving to position (1, 1)
C C C
C D C
C C C

All goals achieved. Cleaning complete.
C C C
C C C
C C C


Cleaning complete.
